In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd  /content/drive/'My Drive'/NYU_Spring_2021/CCM/code/eccentricity

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity


In [176]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import re
import sys
import tarfile

from six.moves import urllib
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

In [178]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


True

In [191]:
def read_tfrecord(examples, labeled=True):
    tfrecord_format = (
        {
            "image_raw": tf.io.FixedLenFeature([], tf.string),
            "label": tf.io.FixedLenFeature([], tf.int64),
        }
    )
    example= tf.io.parse_single_example(examples, tfrecord_format)
    image = tf.io.decode_raw(example["image_raw"], tf.uint8)
    image = tf.cast(image, tf.float32) * (1. / 255)
    image.set_shape(np.prod([5, 60, 60]))
    image = tf.reshape(image, [5, 60, 60, 1])
    image = image - 0.5
    label = tf.cast(example["label"], tf.int32)
    label = tf.one_hot(label, 5)
    # label = tf.keras.utils.to_categorical(y=label, num_classes=5, dtype=tf.int32)
    return image, label

In [192]:
def load_dataset(filename, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filename
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

In [193]:
def get_dataset(filename, labeled=True):
    dataset = load_dataset(filename, labeled=labeled)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(128)
    return dataset

In [194]:
from functools import partial
AUTOTUNE = tf.data.experimental.AUTOTUNE


train_filename = 'experiments/demo-experiment/datasets/gen_fl_0/data/train_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
val_filename = 'experiments/demo-experiment/datasets/gen_fl_0/data/validation_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_filename = 'experiments/demo-experiment/datasets/gen_fl_0/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'

train_dataset = get_dataset(train_filename)
val_dataset = get_dataset(val_filename)
test_dataset = get_dataset(test_filename)


In [197]:
model = keras.Sequential(
    [
        layers.Conv2D(filters=32, kernel_size=5, activation='relu', input_shape=input_shape[1:]), #[128, 5, 56, 56, 32]
        layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2)), # [128, 5, 27, 27, 32]
        layers.Conv2D(filters=32, kernel_size=5, activation='relu'), # [128, 5, 23, 23, 32]
        layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2)), # [128, 5, 11, 11, 32]
        layers.Flatten(), # [128, 19360]
        layers.Dense(num_classes, activation="softmax") # [128, 5]
    ]
)

In [198]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 5, 56, 56, 32)     832       
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 5, 27, 27, 32)     0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 5, 23, 23, 32)     25632     
_________________________________________________________________
max_pooling3d_22 (MaxPooling (None, 5, 11, 11, 32)     0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 19360)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 96805     
Total params: 123,269
Trainable params: 123,269
Non-trainable params: 0
________________________________________________

In [199]:
opt = keras.optimizers.Adagrad(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


In [200]:
epochs = 20
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
)

Epoch 1/2
      4/Unknown - 79s 20s/step - loss: 1.6049 - accuracy: 0.2238

KeyboardInterrupt: ignored

In [201]:
score = model.evaluate(test_dataset, verbose=0)

KeyboardInterrupt: ignored